Code for Simulator - updates excel sheet with Completion status and time

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import random
from datetime import datetime, timedelta, timezone
import os
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from tatc.schemas import TwoLineElements
from tatc.schemas import Point
from tatc.analysis import collect_observations
from tatc import utils
from tatc.schemas import Instrument
from tatc.schemas import WalkerConstellation, SunSynchronousOrbit
from tatc.utils import (
    swath_width_to_field_of_regard,
    along_track_distance_to_access_time,
)
import datetime
from tatc.analysis import collect_multi_observations
from tatc.schemas import Satellite
from datetime import datetime, timedelta, timezone
from tatc.schemas import Point
import logging
logging.basicConfig(level=logging.INFO)
from tatc.analysis import collect_ground_track
from tatc.analysis import compute_ground_track
from tatc.schemas import PointedInstrument, WalkerConstellation, SunSynchronousOrbit
from tatc.utils import swath_width_to_field_of_regard, swath_width_to_field_of_view
import pytz

class Simulator():
    def __init__(self):
        pass

    # This function uses the tatc library to define the  Capella Constellation, we define tle, instrument 
    # Capella const is the final output used further in the function -opportunity
    def const(self):
        roll_angle = (30 + 33.5)/2
        roll_range = (33.5 - 30)
        start = datetime(2018, 1, 1, tzinfo=timezone.utc)
        self.constellation = WalkerConstellation(
            name="SnowGlobe Ku",
            orbit=SunSynchronousOrbit(
                altitude=555e3, 
                equator_crossing_time="06:00:30", 
                equator_crossing_ascending=False,
                epoch=start
            ),
            number_planes=1,
            number_satellites=5,
            instruments=[
                PointedInstrument(
                    name="SnowGlobe Ku-SAR",
                    roll_angle=-roll_angle,
                    field_of_regard=2*roll_angle + swath_width_to_field_of_regard(555e3, 50e3),
                    along_track_field_of_view=swath_width_to_field_of_view(555e3, 50e3, 0),
                    cross_track_field_of_view=roll_range + swath_width_to_field_of_view(555e3, 50e3, roll_angle),
                    is_rectangular=True
                )
            ]
        )
        satellites = self.constellation.generate_members()
        self.satellite_dict = {sat.name: sat for sat in satellites}

    
        # CAPELLA_13_tle= [ 
        # "1 60419U 24142A   24326.56272859  .00014736  00000+0  16272-2 0  9994",
        # "2 60419  53.0086  37.0038 0001423  85.1678 274.9464 14.87633793 15171",
        #     ]
        
        # CAPELLA_14_tle= [ 
        #     "1 59444U 24066C   24326.41256276  .00017846  00000+0  17602-2 0  9997",
        #     "2 59444  45.6101 354.1920 0001417  63.9982 296.1042 14.91842553 26306",
        # ]

        # CAPELLA_15_tle= [ 
        #     "1 60544U 24149CE  24326.63431561  .00009918  00000+0  86496-3 0  9997",
        #     "2 60544  97.7350  40.7817 0005870 174.5984 185.5302 14.97041464 14455",
        # ]

        # CAPELLA_11_tle= [ 
        #     "1 57693U 23126A   24326.55257719  .00001335  00000+0  20605-3 0  9999",
        #     "2 57693  53.0084 132.6964 0004021 153.2687 206.8497 14.79537557 67352",
        # ]

        # CAPELLA_09_tle= [ 
        #     "1 55910U 23035C   24326.19469929  .00080387  00000+0  28081-2 0  9995",
        #     "2 55910  43.9854 265.6943 0016492 143.7069 216.4921 15.28530799 92535",
        # ]

        # CAPELLA_10_tle= [ 
        #     "1 55909U 23035B   24326.33838605  .00062564  00000+0  26624-2 0  9998",
        #     "2 55909  43.9916 278.3602 0014472 119.8274 240.4032 15.22108149 92391",
        # ]

        # capella_09_orbit = TwoLineElements(tle=CAPELLA_09_tle)
        # capella_10_orbit = TwoLineElements(tle=CAPELLA_10_tle)
        # capella_11_orbit = TwoLineElements(tle=CAPELLA_11_tle)
        # capella_13_orbit = TwoLineElements(tle=CAPELLA_13_tle)
        # capella_14_orbit = TwoLineElements(tle=CAPELLA_14_tle)
        # capella_15_orbit = TwoLineElements(tle=CAPELLA_15_tle)

        # # Defining Instrument
        # capella_sar_for = utils.swath_width_to_field_of_regard(500e3, 30e3)
        # capella_sar = Instrument(name="SAR", field_of_regard=capella_sar_for)

        # # Defining Satellite

        # Capella_09 = Satellite(name="Capella 9", orbit=capella_09_orbit, instruments=[capella_sar])
        # Capella_10 = Satellite(name="Capella 10", orbit=capella_10_orbit, instruments=[capella_sar])
        # Capella_11 = Satellite(name="Capella 11", orbit=capella_11_orbit, instruments=[capella_sar])
        # Capella_13 = Satellite(name="Capella 13 ", orbit=capella_13_orbit, instruments=[capella_sar])
        # Capella_14 = Satellite(name="Capella 14", orbit=capella_14_orbit, instruments=[capella_sar])
        # Capella_15 = Satellite(name="Capella 15", orbit=capella_15_orbit, instruments=[capella_sar])

        # # Base Constellation
        # self.Capella_const = [Capella_09,Capella_10,Capella_11,Capella_13,Capella_14,Capella_15]

    # This function reads the master geojson file and filter unprocessed requests
    # (non empty simulation status indicates, that location/request was processed) 
    # key output = self.filtered_req (pandas dataframe with filtered rows )  
    
    def user_request(self):
        self.req = gpd.read_file('Master_file')
        self.filtered_req = self.req[self.req['simulation_status'].isna()]         

    # This function uses tatc library multi - observations to generate the list of observation times
    # It uses all the locations from the user_request function, the simulation time at each step is given as input
    # ie say self.time = 2025-01-01, observations are generated for the next 10 days ( self.time keeps updating with each time step)
    # key output = self.combined_results, is a pandas data frame, column epoch returns the obs time (we sort it ascending)

    def opportunity(self):
        self.const()
        self.user_request()
        end = self._time + timedelta(2)
        combined_results = pd.DataFrame()
        for index,row in self.filtered_req.iterrows():    
            loc = Point(id=row['id'],latitude=row['latitude'],longitude=row['longitude'])
            results = collect_multi_observations(loc, self.constellation, self._time, end)
            combined_results = pd.concat([combined_results, results], ignore_index=True)    
        self.combined_results = combined_results.sort_values(by='epoch', ascending=True)
        
   # whenever the planner uploades geojson file we want requests to be updated
   # (to incorporate changes from optimizer or updates from simulation , ie simulation status updates)
   
    def execute(self,init_time, duration, time_step): 
        # intilization
        self.const()
        self.user_request()
        self._time = self._next_time = self._init_time = init_time
        self._duration = duration
        self._time_step = time_step

        while self._time < self._init_time + self._duration:
            print(f"Current time {self._time}")            
            self._next_time = self._time + self._time_step
            print(f"advancing to {self._next_time}") 

            # updating user requests - read filtered rows
            # Logic if appender completes execution then execute below line
            #self.user_request()         

            # Error handler
            if self.filtered_req.empty:
                logging.info("No observations available. Skipping to next time step.")
                # Can use this condition to reset the master file
                self._time = self._next_time
                continue
            
            self.opportunity() # updating observations list
            self.rs = self.combined_results # writing the observations pandas dataframe to new variable
            self.observation_time = self.rs['epoch'].iloc[0] # latest possible observation            
            self.id = self.rs['point_id'].iloc[0] # point id for the above observation
            self.coord = self.rs['geometry'].iloc[0] # location for the observation
            self.sat = self.rs['satellite'].iloc[0] # satellite collecting the observation
            prev_observation_time = None
            # This below loop is written to handle the time step(1 day), there can be multiple observations within a day 
            # it loops through all the observations possible until the next time step

            while self.observation_time < self._next_time:
                if self.observation_time == prev_observation_time:
                    logging.warning("No progress in observations, breaking loop.")
                    break
                prev_observation_time = self.observation_time
                print(f"Next observation {self.observation_time}") 
                req = self.req # reads the requests file
                #req = pd.read_excel('requests.xlsx')
                req['completion_date'] = pd.to_datetime(req['completion_date'], errors='coerce')  # Ensure completion_date is datetime
                req['simulation_status'] = req['simulation_status'].astype(str)  # Ensure simulation_status is string
                # req['request_status'] = req['request_status'].astype(str)
                req['satellite'] = req['satellite'].astype(str)
                # format time as required in gejson file
                # t = self.observation_time
                t = self.observation_time.replace(tzinfo=None)
                
                # Groundtrack information

                # The below lines updates the excel file for the specific id corresponding to the latest observation time
                # ps note, the locations are not processed sequentially as in excel, id 13 can be processed before id 1 based on satellite location                            

                req.loc[req.id == self.id, 'completion_date'] = t
                req.loc[req.id == self.id, 'simulation_status'] = 'Completed'
                # req.loc[req.id == self.id, 'request_status'] = 'Completed'
                req.loc[req.id == self.id, 'satellite'] = self.sat

                # Groundtrack information
                sat_object = self.satellite_dict.get(self.sat)
                results = collect_ground_track(sat_object,[t],crs='spice')
                req.loc[req.id == self.id, 'geometry'] = results['geometry'].iloc[0]              


                # # Note TO  EMMANUEL 
                # # BELOW LOCATIONS TO BE USED FOR VISUALIZATION

                # lat = self.coord.y
                # lon = self.coord.x
                # print(f"Coordinates - latitude{lat},longitude - {lon}")
                 
                # Save the updated DataFrame back to the Master Geojson file
                req.to_file("Master_file", driver="GeoJSON")
                # Regenerating observations with respect to updated list
                # calling user_request and opprtunity will now exclude the entries processed above(sompleted status) and generate new list
                self.user_request()
                self.opportunity()
                self.rs = self.combined_results
                self.observation_time = self.rs['epoch'].iloc[0]
                self.id = self.rs['point_id'].iloc[0]
                self.coord = self.rs['geometry'].iloc[0]
                self.sat = self.rs['satellite'].iloc[0]
            # simulation advances to next time
            # filter data and write geojson
            self._time = self._next_time     


Executing the simulation

In [4]:
s = Simulator()
from datetime import datetime, timedelta, timezone

start = datetime(2019, 3, 10, tzinfo=timezone.utc)
duration = timedelta(days=5)
time_step = timedelta(days=1)
s.execute(start, duration, time_step)

Current time 2019-03-10 00:00:00+00:00
advancing to 2019-03-11 00:00:00+00:00


C:\Users\dramach6\AppData\Local\Temp\ipykernel_25188\1613146997.py:205: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2019-03-10 00:02:56.166147500+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  req.loc[req.id == self.id, 'completion_date'] = t
INFO:pyogrio._io:Created 51 records


Next observation 2019-03-10 00:02:56.166147500+00:00


KeyError: 'epoch'

Below code not useful for Demo \
Adding this code for understanding the format of observation opportunities \
important columns - epoch, satellite, geometry and point id

In [5]:
start = datetime.now(timezone.utc)
end = start + timedelta(10)
loc = Point(id=1,latitude=45.6541666666667,longitude=-93.6541666666667)
results = collect_multi_observations(loc, s.Capella_const, start, end)
results

,point_id,geometry,satellite,instrument,start,epoch,end,sat_alt,sat_az
0,1,POINT Z (-93.65417 45.65417 0),Capella 14,SAR,2025-01-16 08:58:07.414063+00:00,2025-01-16 08:58:09.408198+00:00,2025-01-16 08:58:11.402333+00:00,88.691463,5.520804
1,1,POINT Z (-93.65417 45.65417 0),Capella 13,SAR,2025-01-16 17:14:09.399685+00:00,2025-01-16 17:14:12.535898+00:00,2025-01-16 17:14:15.672111+00:00,88.929615,72.559161
2,1,POINT Z (-93.65417 45.65417 0),Capella 14,SAR,2025-01-17 09:03:58.057464+00:00,2025-01-17 09:04:00.230898+00:00,2025-01-17 09:04:02.404332+00:00,88.877133,183.549977
3,1,POINT Z (-93.65417 45.65417 0),Capella 14,SAR,2025-01-19 07:34:27.122323+00:00,2025-01-19 07:34:29.479964500+00:00,2025-01-19 07:34:31.837606+00:00,89.340905,7.922122
4,1,POINT Z (-93.65417 45.65417 0),Capella 14,SAR,2025-01-20 07:40:11.014282+00:00,2025-01-20 07:40:13.727604500+00:00,2025-01-20 07:40:16.440927+00:00,89.456202,17.134240
5,1,POINT Z (-93.65417 45.65417 0),Capella 14,SAR,2025-01-22 06:10:27.399790+00:00,2025-01-22 06:10:29.757250500+00:00,2025-01-22 06:10:32.114711+00:00,89.035559,163.889979
6,1,POINT Z (-93.65417 45.65417 0),Capella 13,SAR,2025-01-22 09:53:12.320297+00:00,2025-01-22 09:53:15.214729+00:00,2025-01-22 09:53:18.109161+00:00,89.442544,50.587697
7,1,POINT Z (-93.65417 45.65417 0),Capella 14,SAR,2025-01-23 06:16:05.070217+00:00,2025-01-23 06:16:07.373121500+00:00,2025-01-23 06:16:09.676026+00:00,88.655197,17.434032
8,1,POINT Z (-93.65417 45.65417 0),Capella 15,SAR,2025-01-25 04:22:16.976012+00:00,2025-01-25 04:22:19.056527500+00:00,2025-01-25 04:22:21.137043+00:00,89.197269,66.564930


In [7]:
import tatc
print(tatc.__version__)


3.2.1


In [3]:
master = gpd.read_file('Master_file')
filtered_req = master[master['simulation_status'].isna()] 
filtered_req

,id,time,final_eta,Planner_geometry,centroid,latitude,longitude,expiration_date,simulation_status,completion_date,satellite,geometry


SnowGlobe Constellation